In [7]:
import cv2
import numpy as np

net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

distance_thres = 50
image = cv2.imread("dog.png")

In [8]:
height, width = image.shape[:2]

blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

net.setInput(blob)
layer_names = net.getUnconnectedOutLayersNames()
outs = net.forward(layer_names)

In [9]:
classes = [] # classes list
with open("coco.names", "r") as f:
  classes = [line.strip() for line in f.readlines()] 
print(classes)
colors = np.random.uniform(0, 255, size=(len(classes), 3))

img = image

height, width, channels = image.shape 
class_ids = []
confidences = []   
boxes = []
for out in outs:
  for detection in out: 
      scores = detection[5:]  
      class_id = np.argmax(scores)
      confidence = scores[class_id] 
      if confidence > 0.2:
          # Object detected
          center_x = int(detection[0] * width)
          center_y = int(detection[1] * height) 
          w = int(detection[2] * width)
          h = int(detection[3] * height) 
          # Rectangle coordinates
          x = int(center_x - w / 2)
          y = int(center_y - h / 2)
          boxes.append([x, y, w, h]) 
          confidences.append(float(confidence)) 
          class_ids.append(class_id) 
          

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.35,0.4)
font = cv2.FONT_HERSHEY_PLAIN 
for i in range(len(boxes)):
  if i in indexes:
      x, y, w, h = boxes[i]
      label = str(classes[class_ids[i]])
      color = colors[i]
      cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
      cv2.circle(image, (x + w // 2, y + h // 2), 5, (0, 0, 255), 2)
      cv2.putText(img, label, (x, y + 30), font, 3, color, 2)

cv2.imshow("classification",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
